In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import datetime as dt
import math
from sklearn.preprocessing import StandardScaler
import helper
from helper import *

In [2]:
df = load_and_preprocess()
df = df.reset_index(drop = True)

In [3]:
df.shape

(45252, 16)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45252 entries, 0 to 45251
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   datetime        45252 non-null  object 
 1   Description     45252 non-null  object 
 2   Humidity        45252 non-null  float64
 3   Wind Direction  45252 non-null  float64
 4   Temperature     45252 non-null  float64
 5   Pressure        45252 non-null  float64
 6   Wind Speed      45252 non-null  float64
 7   clouds          45252 non-null  int64  
 8   rain            45252 non-null  int64  
 9   mist            45252 non-null  int64  
 10  snow            45252 non-null  int64  
 11  shower          45252 non-null  int64  
 12  thunderstorm    45252 non-null  int64  
 13  fog             45252 non-null  int64  
 14  other           45252 non-null  int64  
 15  Intensity       45252 non-null  float64
dtypes: float64(6), int64(8), object(2)
memory usage: 5.5+ MB


In [5]:
df['datetime']= pd.to_datetime(df['datetime'])

#### Just Using Date/Previous Hour Temp.

In [6]:
df.head()

,datetime,Description,Humidity,Wind Direction,Temperature,Pressure,Wind Speed,clouds,rain,mist,snow,shower,thunderstorm,fog,other,Intensity
0,2012-10-01 13:00:00,heavy clouds,93.0,1001.0,285.830000,230.0,4.0,1,0,0,0,0,0,0,0,3.0
1,2012-10-01 14:00:00,sky is clear,91.0,986.0,285.834650,230.0,4.0,0,0,0,0,0,0,0,0,0.0
2,2012-10-01 15:00:00,sky is clear,87.0,945.0,285.847790,231.0,4.0,0,0,0,0,0,0,0,0,0.0
3,2012-10-01 16:00:00,sky is clear,84.0,904.0,285.860929,233.0,4.0,0,0,0,0,0,0,0,0,0.0
4,2012-10-01 17:00:00,sky is clear,80.0,863.0,285.874069,234.0,3.0,0,0,0,0,0,0,0,0,0.0


In [7]:
df['day'] = df['datetime'].dt.day
df['month'] = df['datetime'].dt.month
df['year'] = df['datetime'].dt.year

df['lag1'] = df['Temperature'].shift(periods = 1, fill_value = 0)
df['lag2'] = df['Temperature'].shift(periods = 2, fill_value = 0)
df['lag12'] = df['Temperature'].shift(periods = 12, fill_value = 0)
df['lag30'] = df['Temperature'].shift(periods = 30, fill_value = 0)

df['avg_returns'] = 0 
for i in range(1, len(df)):
    df['avg_returns'][i] = df['lag1'][:i+1].mean()
    
df.drop('datetime', axis = 1, inplace = True)
df.drop('Description', axis = 1, inplace = True)

#we have to eliminate all those with 0s
df = df[df['lag30'] != 0]
df

C:\Users\Steven\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Humidity,Wind Direction,Temperature,Pressure,Wind Speed,clouds,rain,mist,snow,shower,...,other,Intensity,day,month,year,lag1,lag2,lag12,lag30,avg_returns
30,71.861538,1016.424953,288.94,282.0,2.0,0,0,0,0,0,...,0,0.0,2,10,2012,288.94,288.69,286.058023,285.830000,277
31,71.861538,1016.424953,288.69,239.0,5.0,0,0,0,0,0,...,0,0.0,2,10,2012,288.94,288.94,286.071162,285.834650,277
32,77.000000,1007.000000,288.55,260.0,7.0,1,0,0,0,0,...,0,3.0,2,10,2012,288.69,288.94,286.084302,285.847790,277
33,71.861538,1016.424953,288.22,284.0,0.0,0,0,0,0,0,...,0,0.0,2,10,2012,288.55,288.69,286.097442,285.860929,278
34,71.861538,1016.424953,287.92,267.0,0.0,0,0,0,0,0,...,0,0.0,2,10,2012,288.22,288.55,286.110581,285.874069,278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45247,64.000000,1021.000000,275.13,300.0,10.0,1,0,0,0,0,...,0,2.0,29,11,2017,276.45,276.15,275.380000,265.120000,280
45248,59.000000,1023.000000,274.13,300.0,7.0,1,0,0,0,0,...,0,2.0,29,11,2017,275.13,276.45,277.450000,266.120000,280
45249,66.000000,1024.000000,273.48,290.0,7.0,1,0,0,0,0,...,0,1.0,29,11,2017,274.13,275.13,278.770000,268.100000,280
45250,58.000000,1026.000000,272.48,290.0,4.0,1,0,0,0,0,...,0,1.0,29,11,2017,273.48,274.13,278.480000,269.770000,280
